In [1]:
import os
from datetime import datetime
import time
import pyaudio
import librosa
import asyncio
import threading
#import deepspeech
import wave
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, AutoProcessor, AutoModelForCTC, TFWav2Vec2ForCTC, \
                         AutoTokenizer, AutoFeatureExtractor, T5ForConditionalGeneration, T5Tokenizer
import spacy
from spellchecker import SpellChecker
#from autocorrect import Speller
import openai

import torch
import torchaudio
import tensorflow

## ПРЕДИСЛОВИЕ

#### Перед тем, как запускать прогу, необходимо:

1) установить virtual audio cable

2) в sound mixer options для всех бырать defaul input = Linme 1(virtual audio cable)

3) затем для нужной программы (окна) в oputput выбрать тот же кабель (Line 1, например) 

4) вызвать код для нахождения подключённых аудиоустройств: ```for i in range(a.recorder.get_device_count()):
       print(i, a.recorder.get_device_info_by_index(i))```

5) выбрать подходящее аудиоустройство (для записи должно быть устройство Line 1, у которого ```'maxInputChannels' != 0```,
  а для вывода должно быть устройство Line 1, у которого ```'maxOutputChannels' != 0```

6) и когда открываете потоки в kwargs передаёте нужный вам индекс в переменную ```input_device_index```

7) после запуска у нас звук будет перенаправляться так: приложение->Line_1_output->Line_1_input->Наша прога.

In [2]:
class AudioChanel():
    def __init__(self):
        #change this according to your model
        self.chunk = 1024
        self.format = pyaudio.paInt16
        self.channels = 1#2
        self.rate = 16000
        self.recorder = pyaudio.PyAudio()
        self.frames = []
        self.seconds = 2
        # Открыть поток для записи
        #1 {'index': 1, 'structVersion': 2, 'name': 'Line 1 (Virtual Audio Cable)', 'hostApi': 0, 'maxInputChannels': 2, 
        #   'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 
        #   'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
        self.input_stream = self.recorder.open(format=self.format,
                                          channels=self.channels,
                                          rate=self.rate,
                                          input=True,
                                          frames_per_buffer=self.chunk,
                                          input_device_index=1) #0,1 - work
        # Открыть поток для воспроизведения
        #5 {'index': 5, 'structVersion': 2, 'name': 'Line 1 (Virtual Audio Cable)', 'hostApi': 0, 'maxInputChannels': 0, 
        #   'maxOutputChannels': 2, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 
        #   'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
        self.output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        
        
    def record(self):
        while True:
            data = self.input_stream.read(self.chunk)
            self.frames.append(data)

    def play(self, audio = None):
        output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        match audio:
            case list():
                for frame in audio:
                    output_stream.write(frame)
            case _:
                output_stream.write(audio.tobytes())

    def play_default(self):
        output_stream = self.recorder.open(format=self.format,
                                           channels=self.channels,
                                           rate=self.rate,
                                           output=True,
                                           frames_per_buffer=self.chunk,
                                           input_device_index=5) #5- work
        with threading.Lock():
            to_play = list(self.frames)
        for frame in to_play:
            output_stream.write(frame)
    
    def get(self):
        with threading.Lock():
            return list(self.frames)
        
        
    def close(self):
        self.input_stream.stop_stream()
        self.input_stream.close()

In [3]:
class Agent():
    def __init__(self):
        self.audio = AudioChanel()
        self.device = torch.device('cuda')
        #you may use your models instead
        self.processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
        self.tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
        self.model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h").to(self.device)
        self.feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

        self.spell = SpellChecker()
        self.nlp = spacy.load('en_core_web_sm')#en_core_web_trf
        self.PASS_VAR = False

        #the worst model EVER
        model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'# small, large and base
        self.model_translate = T5ForConditionalGeneration.from_pretrained(model_name).to(self.device)
        self.tokenizer_translate = T5Tokenizer.from_pretrained(model_name)
        
        language = 'ru'
        model_id = 'v4_ru'
        self.sample_rate = 48000
        self.speaker = 'kseniya'
        
        self.model_voice, _ = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                             model='silero_tts',
                                             language=language,
                                             speaker=model_id)
        self.model_voice.to(self.device)
        self.to_play = []
    
    def correcting(self,sent):
        to_spell = sent.split()
        corrected_words = [self.spell.correction(word) for word in to_spell]
        return ' '.join((word if word is not None else to_spell[idx]) for idx, word in enumerate(corrected_words))

    def send_to_play(self):
        while True:
            time.sleep(1)
            with threading.Lock():
                copy_to_play = list(self.to_play)
            while len(copy_to_play):
                to_play = copy_to_play[0]
                _ = threading.Thread(target=self.audio.play,args=(to_play[0],).start()
                _ = threading.Thread(target=self.audio.play,args=(to_play[1],).start()
                with threading.Lock():
                    del self.to_play[0]
                    del copy_to_play[0]
    
    async def run(self):
        #start recording
        _ = threading.Thread(target=self.audio.record).start()
        _ = threading.Thread(target=self.send_to_play).start()
        
        # await asyncio.sleep(10)
        last_state = ''
        while True:
            '''
            PART 1
            RECORDING
            description: record every 2 seconds of audio from PC
            '''
            await asyncio.sleep(2)
            time = datetime.now()
            frames = self.audio.get()[:200]
            to_model = np.frombuffer(b''.join(frames), dtype=np.int16)
            end_1 = datetime.now()
            # print('end_1: ', end_1)
            '''
            PART 2
            CONVERTING AUDIO TO TEXT
            description: put audio into model and get recognized words
            '''
            input_values = self.processor(to_model, sampling_rate = self.audio.rate, return_tensors="pt").input_values  # Batch size 1
            logits = self.model(input_values.to(torch.float).to(self.device)).logits
            predicted_ids = torch.argmax(logits, axis=-1)
            
            outputs = self.tokenizer.decode(predicted_ids[0], output_word_offsets=True)
            # compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
            time_offset = self.model.config.inputs_to_logits_ratio / self.feature_extractor.sampling_rate
            
            word_offsets = [
                {
                    "word": d["word"],
                    "start_time": round(d["start_offset"] * time_offset, 2),
                    "end_time": round(d["end_offset"] * time_offset, 2),
                }
                for d in outputs.word_offsets
            ]
            end_2 = datetime.now()
            print('end_2: ', end_2-end_1)
            '''
            PART 3
            FIND SENTENCIES IN THE OUTPUT TEXT
            description: find grammar parts in output text for building whole sentences
            '''
            text = ' '.join([word['word'] for word in word_offsets])
            fixed = self.correcting(text)
            print('Full recorded text: ', text)
            if last_state == '':
                last_state = text
            else:
                if last_state == text:
                    self.PASS_VAR = True
                last_state = text
                
            doc = self.nlp(fixed)
    
            end_3 = datetime.now()
            print('end_3: ', end_3-end_2)
            '''
            PART 4
            TRANSLATE SENTENCE INTO RUSSIAN (or any other language)
            description: translate one of the list, because we should be sure about full context
            '''
            sents = list(doc.sents)
            if len(sents)<=1 and not self.PASS_VAR:
                continue
            self.PASS_VAR = False
            print(sents)
            
            prefix = 'translate to ru: '
            ### delete part of audio.frames list
            with threading.Lock():
                idx_del = len(sents[0].text.split())-1 if len(sents[0].text.split())==len(word_offsets) else len(sents[0].text.split())
                to_del = int(word_offsets[idx_del]['end_time']*self.audio.rate/self.audio.chunk)
                play_background = self.audio.frames[:to_del+1]
                del self.audio.frames[:to_del]
            ###
            src_text = prefix + sents[0].text
            
            # translate Englosh to Russian
            input_ids = self.tokenizer_translate(src_text, return_tensors="pt")
            
            generated_tokens = self.model_translate.generate(**input_ids.to(self.device))
            
            result = self.tokenizer_translate.batch_decode(generated_tokens, skip_special_tokens=True)
            end_4 = datetime.now()
            print('end_4: ', end_4-end_3)
            '''
            PART 5
            VOICEOVER OF TEXT AND PLAY
            description: voice translated text
            '''
            print(result[0])
            if not len(result[0]):
                continue
                  
            audio = self.model_voice.apply_tts(text=result[0],
                        speaker=self.speaker,
                        sample_rate=self.sample_rate)
            
            audio_numpy = audio.cpu().numpy()
            audio_resampled = librosa.resample(audio_numpy, orig_sr=self.sample_rate, 
                                               target_sr=self.audio.rate, res_typestr='fft')
            audio_int16 = (audio_resampled * 32767).astype(np.int16)
            end_5 = datetime.now()
            print('end_5: ', end_5-end_4)
            # self.audio.play(audio_int16)
            with threading.Lock():
                self.to_play.append((audio_int16,frames))
            # _ = threading.Thread(target=self.audio.play, args=(play_background,)).start()
            
a = Agent()
await a.run()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3272317480.py, line 44)

In [5]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to('cuda')

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device='cuda',
)

await asyncio.sleep(10)
l = a.audio.get() #данные из потока pyaudio
to_model = np.frombuffer(b''.join(l), dtype=np.int16)
result = pipe(l)
print(result)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[]


In [21]:
model_name = 'utrobinmv/t5_translate_en_ru_zh_large_1024'
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to('cuda')
tokenizer = T5Tokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [55]:
spell = SpellChecker()

# find those words that may be misspelled
# misspelled = spell.unknown(['something', 'is', 'hapenning', 'here','booglaunch'])
to_spell = ['something', 'is', 'hapenning', 'here','booglaunch']
corrected_words = [spell.correction(word) for word in to_spell]
fixed = ' '.join((word if word is not None else to_spell[idx]) for idx, word in enumerate(corrected_words))

prefix = 'translate to ru: '
src_text = prefix + fixed

# translate Russian to Chinese
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids.to('cuda'))

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)

language = 'ru'
model_id = 'v4_ru'
sample_rate = 48000
speaker = 'kseniya'
device = torch.device('cuda')

model_voice, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model_voice.to(device)  # gpu or cpu

audio = model_voice.apply_tts(text=result[0],
                        speaker=speaker,
                        sample_rate=sample_rate)
audio

['Что-то здесь происходит, бугзапуск']


Using cache found in C:\Users\User/.cache\torch\hub\snakers4_silero-models_master


tensor([-5.3106e-03, -7.0865e-03, -7.2111e-03,  ...,  2.8435e-05,
         9.6688e-06, -7.1262e-06])

In [57]:
import librosa
# Переводим тензор PyTorch в numpy массив
audio_numpy = audio.cpu().numpy()
audio_resampled = librosa.resample(audio_numpy, orig_sr=48000, target_sr=16000, res_typestr='fft')
# Нормализуем данные и конвертируем их в формат int16 для pyaudio
audio_int16 = (audio_resampled * 32767).astype(np.int16)
p = pyaudio.PyAudio()

# Открытие потока для вывода
stream = p.open(format=pyaudio.paInt16, 
                channels=1, 
                rate=16000, 
                output=True)

# Воспроизведение аудио
stream.write(audio_int16.tobytes())

# Закрытие потока
stream.stop_stream()
stream.close()

In [62]:
# set cache for hugging face
os.environ["HF_DATASETS_CACHE"] = "E:\cache"
os.environ["TRANSFORMERS_CACHE"] = "E:\cache"
os.environ['HF_HOME'] = "E:\cache"

In [4]:
#TODO когда санкции снимут
# import os
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./google_cred.json"

In [5]:
# Загрузка предобученной модели и процессора
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You s

In [4]:
model_base = AutoModelForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm")
processor_base = AutoProcessor.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm")

Some weights of the model checkpoint at patrickvonplaten/wav2vec2-base-100h-with-lm were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at patrickvonplaten/wav2vec2-base-100h-with-lm and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You s

C:\Users\User\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--patrickvonplaten--wav2vec2-base-100h-with-lm. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


C:\Users\User\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\pyctcdecode\models--patrickvonplaten--wav2vec2-base-100h-with-lm. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


NameError: name 'kenlm' is not defined

## Initialize your models

In [3]:
processor_l960 = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer_l960 = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model_l960 = TFWav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor_l960 = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

nlp = spacy.load('en_core_web_sm')

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

E:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\cache\hub\models--facebook--wav2vec2-base-960h. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

E:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]


TFWav2Vec2ForCTC has backpropagation operations that are NOT supported on CPU. If you wish to train/fine-tune this model, you need a GPU or a TPU
All PyTorch model weights were used when initializing TFWav2Vec2ForCTC.

Some weights or buffers of the TF 2.0 model TFWav2Vec2ForCTC were not initialized from the PyTorch model and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
async def main(processor, model, tokenizer, feature_extractor, nlp):
    a = AudioChanel()
    frames = await a.recording()
    to_model = np.frombuffer(b''.join(frames), dtype=np.int16)
    
    
    device = torch.device('cuda')
    input_values = processor(to_model, sampling_rate = a.rate, return_tensors="tf").input_values  # Batch size 1
    logits = model(input_values).logits
    predicted_ids = tensorflow.argmax(logits, axis=-1)

    # transcription = processor.decode(predicted_ids[0])
    # print("Transcription:", type(transcription), transcription)
    
    outputs = tokenizer.decode(predicted_ids[0], output_word_offsets=True)
    # compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
    time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate
    
    word_offsets = [
        {
            "word": d["word"],
            "start_time": round(d["start_offset"] * time_offset, 2),
            "end_time": round(d["end_offset"] * time_offset, 2),
        }
        for d in outputs.word_offsets
    ]
    print(word_offsets)

    text = ' '.join([word['word'] for word in word_offsets])
    print('Full recorded text: ', text)
    doc = nlp(text)
    for sent in doc.sents:
        print("Sentence:", sent.text)

#     outputs = processor.decode(logits, output_word_offsets=True)
#     time_offset = model.config.inputs_to_logits_ratio / processor.feature_extractor.sampling_rate
#     word_offsets = [
#         {
#             "word": d["word"],
#             "start_time": round(d["start_offset"] * time_offset, 2),
#             "end_time": round(d["end_offset"] * time_offset, 2),
#         }
#         for d in outputs.word_offsets
#     ]
#     print(word_offsets)
#     model_path = './deepspeech-0.9.3-models.pbmm'
#     model = deepspeech.Model(model_path)
#     print(model.stt(to_model))
    
    a.close()
    
# await main(processor, model)
await main(processor_l960, model_l960, tokenizer_l960, feature_extractor_l960, nlp)

[{'word': 'AH', 'start_time': 1.42, 'end_time': 1.5}, {'word': 'BUT', 'start_time': 1.98, 'end_time': 2.12}, {'word': 'INSTEAD', 'start_time': 2.62, 'end_time': 2.94}, {'word': 'OF', 'start_time': 3.0, 'end_time': 3.04}, {'word': 'MISS', 'start_time': 3.28, 'end_time': 3.44}, {'word': 'BUSSING', 'start_time': 3.48, 'end_time': 3.78}, {'word': 'CAUSE', 'start_time': 3.84, 'end_time': 4.0}, {'word': 'THIS', 'start_time': 4.1, 'end_time': 4.2}, {'word': 'IS', 'start_time': 4.28, 'end_time': 4.32}, {'word': 'A', 'start_time': 4.36, 'end_time': 4.38}, {'word': 'BOOGLAUNCH', 'start_time': 4.44, 'end_time': 4.86}, {'word': 'IS', 'start_time': 4.92, 'end_time': 5.0}, {'word': 'NOT', 'start_time': 5.04, 'end_time': 5.14}, {'word': 'ACTUALLY', 'start_time': 5.22, 'end_time': 5.44}, {'word': 'AT', 'start_time': 5.5, 'end_time': 5.54}, {'word': 'ALL', 'start_time': 5.68, 'end_time': 5.78}, {'word': 'WITH', 'start_time': 5.82, 'end_time': 5.9}, {'word': 'UTS', 'start_time': 5.96, 'end_time': 6.02},

In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer

class TextCorrector:
    def __init__(self, model_name='facebook/bart-large-cnn'):
        self.tokenizer = BartTokenizer.from_pretrained(model_name)
        self.model = BartForConditionalGeneration.from_pretrained(model_name)
        self.model.eval()  # Перевести модель в режим оценки
        if torch.cuda.is_available():
            self.model.to('cuda')

    def correct_text(self, text):
        inputs = self.tokenizer.encode(text, return_tensors='pt', max_length=1024, truncation=True)
        if torch.cuda.is_available():
            inputs = inputs.to('cuda')
        
        with torch.no_grad():
            outputs = self.model.generate(inputs, max_length=1024, num_beams=4, early_stopping=True)
        corrected_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return corrected_text

# Инициализация корректора текста
corrector = TextCorrector()

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

E:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\cache\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

E:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [1]:
from gramformer import Gramformer
grammar_correction = Gramformer(models = 1, use_gpu=True)
phrases = [text.lower()]
for phrase in phrases:
  corrections = grammar_correction.correct(phrase, max_candidates=2)
  print(f'[Incorrect phrase] {phrase}')
  for i in range(len(corrections)):
    print(f'[Suggestion #{i}] {corrections[i]}')

E:\anaconda\Lib\site-packages\transformers\models\auto\tokenization_auto.py:769: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
E:\anaconda\Lib\site-packages\transformers\models\auto\auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


pytorch_model.bin:  61%|######1   | 545M/892M [00:00<?, ?B/s]

E:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\cache\hub\models--prithivida--grammar_error_correcter_v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


[Gramformer] Grammar error correct/highlight model loaded..


NameError: name 'text' is not defined

In [10]:
text = "AH BUT INSTEAD OF MISS BUSSING CAUSE THIS IS A BOOGLAUNCH IS NOT ACTUALLY AT ALL WITH UTS OF THINGS I WILL SAVE THAT FHE NEXT SHE IS TALL TO SEE YOU THERE HE'S GOING"
phrases = [text.lower()]
# phrases = ['BOOGLAUNCH it is a book launch']

for phrase in phrases:
    corrections = grammar_correction.correct(phrase, max_candidates=5)
    print(f'[Incorrect phrase] {phrase}')
    for i in corrections:
        print(f'[Suggestion #] {i}]')

[Incorrect phrase] ah but instead of miss bussing cause this is a booglaunch is not actually at all with uts of things i will save that fhe next she is tall to see you there he's going
[Suggestion #] ah but instead of miss bussing cause this is a booglaunch is not actually at all with uts of things i will save that fhe next she is tall to see you there he's going to be able to see it.]
[Suggestion #] ah but instead of miss bussing cause this is a booglaunch is not actually at all with uts of things i will save that fhe next she is tall to see you there he's going to be able to see if he']
[Suggestion #] ah but instead of miss bussing cause this is a booglaunch is not actually at all with uts of things i will save that fhe next she is tall to see you there he's going to be able to see.]
[Suggestion #] ah but instead of miss bussing cause this is a booglaunch is not actually at all with uts of things i will save that fhe next she is tall to see you there he's going to be able to see if i

In [9]:
text = "AH BUT INSTEAD OF MISS BUSSING CAUSE THIS IS A BOOGLAUNCH IS NOT ACTUALLY AT ALL WITH UTS OF THINGS I WILL SAVE THAT FHE NEXT SHE IS TALL TO SEE YOU THERE HE'S GOING"
text_accumulated = "translate text into russian: " + text.lower()
corrector.correct_text('i hate machine learning')
# inputs = corrector.tokenizer('say hi', return_tensors="pt", max_length=512, truncation=True).to('cuda')
    
# # Получение предсказаний от модели
# with torch.no_grad():
#     outputs = corrector.model(**inputs)

# # Получаем логиты токенов
# logits = outputs.logits

# # Выбираем самые вероятные токены
# predicted_tokens = torch.argmax(logits, dim=-1)

# # Преобразуем токены обратно в текст
# corrected_text = corrector.tokenizer.decode(predicted_tokens[0])
# print(corrected_text)
# Отправка накопленного текста на коррекцию
# corrected_text = corrector.correct_text(text_accumulated)
# print("Corrected Text:", corrected_text)

"i hate machine learning. i hate machinelearning. I hatemachine learning. I love machine learning, but I hate it when it tries to make me do something I don't want to do. I. hate machineLearning. I Hate Machine Learning. I Love Machine Learning, I Love You."

In [104]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation',model='gpt2-large')
set_seed(42)
generator("BOOGLAUNCH is", max_length=32, num_return_sequences=1)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "BOOGLAUNCH is an acronym that means you can easily find information about everything in a single word with no words, and there's a great tutorial for"}]

In [105]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key='w',
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model="gpt-3.5-turbo-16k",
)

PermissionDeniedError: Error code: 403 - {'error': {'code': 'unsupported_country_region_territory', 'message': 'Country, region, or territory not supported', 'param': None, 'type': 'request_forbidden'}}

In [3]:
a = AudioChanel()
for i in range(a.recorder.get_device_count()):
    print(i, a.recorder.get_device_info_by_index(i))#['name'].encode('latin1', errors='ignore').decode('cp1251'))

0 {'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
1 {'index': 1, 'structVersion': 2, 'name': 'Line 1 (Virtual Audio Cable)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
2 {'index': 2, 'structVersion': 2, 'name': 'РњРёРєСЂРѕС„РѕРЅ (Realtek(R) Audio)', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
3 {'index': 3, 'structVersion': 2, 'name': 'Line 2 (Virtual Audio Cable)', 'hostApi': 0, 'maxInpu